In [1]:
from math import sqrt
import pandas as pd
import numpy as np

from pathlib import Path
from collections import defaultdict
import pickle
import gc

from tqdm import tqdm
from pandarallel import pandarallel

from scipy.sparse import csr_matrix, coo_matrix
import implicit
import catboost

import sys
sys.path.append("..")
from src.utils import *
from src.dataset import *
from src.trending import *

from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from catboost import Pool, CatBoostClassifier, cv
from catboost.utils import get_roc_curve, create_cd
from catboost.eval.catboost_evaluation import CatboostEvaluation

pd.set_option('display.max_colwidth', 255)
tqdm.pandas()
pandarallel.initialize(progress_bar=True, nb_workers=8, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# TRAIN 

In [2]:
merged_data = pd.read_csv("../tmp/purch_popular_pairs_train.csv", index_col=False)
merged_data = merged_data.fillna(-1).astype(np.int32)
merged_data["merged"] = 1
merged_data

,customer_id,article_id,purchase_score,general_popular_count,general_popular_count_rank,general_popular_trending_sum,general_popular_trending_sum_rank,group_popular_trending_mean,group_popular_trending_mean_rank,group_popular_trending_sum,group_popular_trending_sum_rank,group_popular_count,group_popular_count_rank,pairs_parent_purchase_score,merged
0,0,16003,330,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,28864,1
1,0,16003,330,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,1
2,0,16023,28864,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,0,65667,48,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
4,0,78719,7,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92345332,1218691,93412,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,108770,1
92345333,1219679,28970,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,95755,1
92345334,1238517,95228,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,54385,1
92345335,1359561,95228,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,12707,1


In [3]:
similar_data = pd.read_csv("../tmp/als_similarity_train.csv", index_col=False)
similar_data = similar_data.fillna(-1).astype(np.int32)
similar_data["similar"] = 1
similar_data

,customer_id,similar_parent_purchase_score,article_id,als_similarity,similar
0,0,0,1300,16,1
1,0,0,2912,16,1
2,0,0,2060,14,1
3,0,0,1309,13,1
4,0,0,61076,8,1
...,...,...,...,...,...
135505735,1371893,0,270,9,1
135505736,1371893,0,76810,5,1
135505737,1371893,0,99975,5,1
135505738,1371893,0,93787,4,1


In [4]:
train_data_left = merged_data.merge(similar_data, on=["customer_id", "article_id"], how='left')
train_data_left = train_data_left.fillna(-1).astype(np.int32)
train_data_left

,customer_id,article_id,purchase_score,general_popular_count,general_popular_count_rank,general_popular_trending_sum,general_popular_trending_sum_rank,group_popular_trending_mean,group_popular_trending_mean_rank,group_popular_trending_sum,group_popular_trending_sum_rank,group_popular_count,group_popular_count_rank,pairs_parent_purchase_score,merged,similar_parent_purchase_score,als_similarity,similar
0,0,16003,330,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,28864,1,330.0,212.0,1.0
1,0,16003,330,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,6,1,330.0,212.0,1.0
2,0,16023,28864,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,330.0,39.0,1.0
3,0,16023,28864,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,28864.0,98.0,1.0
4,0,16023,28864,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,6.0,62.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96593140,1218691,93412,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,108770,1,NaN,NaN,NaN
96593141,1219679,28970,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,95755,1,NaN,NaN,NaN
96593142,1238517,95228,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,54385,1,NaN,NaN,NaN
96593143,1359561,95228,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,12707,1,NaN,NaN,NaN


In [5]:
train_data_right = merged_data.merge(similar_data, on=["customer_id", "article_id"], how='right')
train_data_right = train_data_right.fillna(-1).astype(np.int32)
train_data_right

,customer_id,article_id,purchase_score,general_popular_count,general_popular_count_rank,general_popular_trending_sum,general_popular_trending_sum_rank,group_popular_trending_mean,group_popular_trending_mean_rank,group_popular_trending_sum,group_popular_trending_sum_rank,group_popular_count,group_popular_count_rank,pairs_parent_purchase_score,merged,similar_parent_purchase_score,als_similarity,similar
0,0,1300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,16,1
1,0,2912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,16,1
2,0,2060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,14,1
3,0,1309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,13,1
4,0,61076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136232141,1371893,270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9,1
136232142,1371893,76810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5,1
136232143,1371893,99975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,5,1
136232144,1371893,93787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,4,1


In [6]:
del merged_data, similar_data
gc.collect()

0

In [ ]:
train_data_right[train_data_right["merged"].isna()]

In [13]:
(merged_data.memory_usage(deep=True).sum() + similar_data.memory_usage(deep=True).sum()) / 1024**3

7.523404002189636

In [2]:
SEED = 1
N = 12
TEST_ON = 1

cv_iteration = 0

dataset = Dataset(skip_days=7 * cv_iteration, test_days=7 * TEST_ON)
train, test = dataset.get_train_and_test()
articles = dataset.get_articles()
customers = dataset.get_customers()
print("Dataset created")

del train
gc.collect()

Dataset created


0

In [3]:
# purch_data = pd.read_csv("../tmp/purchase_data_train.csv", index_col=False, dtype=np.uint32)
# popular = pd.read_csv("../tmp/popular_train.csv", index_col=False)#.fillna(-9999).astype(np.int32)
# pairs_purch_data = pd.read_csv("../tmp/pairs_train.csv", index_col=False, dtype=np.uint32)
# similar_purch_data = pd.read_csv("../tmp/als_similarity_train.csv", index_col=False, dtype=np.uint32)
# article_factors = pd.read_csv("../tmp/article_factors_train.csv", index_col=False)
# customer_factors = pd.read_csv("../tmp/customer_factors_train.csv", index_col=False)

In [3]:
purch_data = pd.read_csv("../tmp/purchase_data_actual_train.csv", index_col=False, dtype=np.uint32)
popular = pd.read_csv("../tmp/popular_train.csv", index_col=False).fillna(-1).astype(np.int32)
train_data = purch_data.merge(popular, on=["customer_id", "article_id"], how='outer')
del purch_data, popular
gc.collect()

train_data = train_data.fillna(-1).astype(np.int32).astype({
    "general_popular_count_rank": np.int8,
    "group_popular_trending_mean_rank": np.int8,
    "group_popular_count_rank": np.int8
})

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'general_popular_trending_sum_rank' not found in columns."

In [6]:
pairs_purch_data = pd.read_csv("../tmp/pairs_train.csv", index_col=False, dtype=np.uint32)
train_data = train_data.merge(pairs_purch_data, on=["customer_id", "article_id"], how='outer')
del pairs_purch_data
gc.collect()

train_data = train_data.fillna(-1).astype(np.int32).astype({
    "general_popular_count_rank": np.int8,
    "group_popular_trending_mean_rank": np.int8,
    "group_popular_count_rank": np.int8
})

In [ ]:
similar_purch_data = pd.read_csv("../tmp/als_similarity_train.csv", index_col=False, dtype=np.uint32)
train_data = train_data.merge(similar_purch_data, on=["customer_id", "article_id"], how='outer')
del similar_purch_data
gc.collect()

train_data = train_data.fillna(-1).astype(np.int32).astype({
    "general_popular_count_rank": np.int8,
    "group_popular_trending_mean_rank": np.int8,
    "group_popular_count_rank": np.int8
})

In [ ]:
train_data